# Visualization notebook

This notebook is meant for visualizing stuff and testing code. 

In [5]:
import numpy as np
import pandas as pd
import os
import warnings
from dataclasses import dataclass
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from contextlib import contextmanager
from typing import Generator
import demoji
warnings.filterwarnings("ignore", category=FutureWarning)

import spacy
from spacy.tokens import Doc
from spacy import Language, displacy
from vectorizer import nlp


In [8]:

def _load_data(data_dir:str) -> pd.DataFrame:
    """Loads a directory of .jsonl files"""
    return pd.read_json(data_dir, lines=True)
    
    
pan_docs = _load_data("data/pan22/preprocessed/pan22_preprocessed.jsonl")["fullText"].to_list()


test_docs = [
    "It was Jane's car that got stolen last night."
]

doc = nlp(test_docs[0])

for token in doc:
    print(token.text, token.dep_,  [child for child in token.children])
    
displacy.render(doc)

It nsubj []
was ROOT [It, car, .]
Jane poss ['s]
's case []
car attr [Jane, stolen]
that nsubjpass []
got auxpass []
stolen relcl [that, got, night]
last amod []
night npadvmod [last]
. punct []


[]

In [5]:
from verbalizer import Verbalizer

datapaths = {
        "hrs": "data/hrs_release_03-20-23/raw/hrs1_03-20-23_background_bgg_350_anonymized.jsonl",
        "pan":"data/pan22/preprocessed/pan22_preprocessed.jsonl"
    }
    
pan_verb = Verbalizer(datapaths["pan"])

In [8]:
pan_ids = {
    "authors":[
        "en_110",
        "en_100",
        "en_113"
        ],
    "docs":[
        "ed5ec66c-d70f-11ed-8cc6-76349838619d",
        "ed73dd7c-d70f-11ed-8cc6-76349838619d",
        "ee46a040-d70f-11ed-8cc6-76349838619d"
        ]
    }

# API will change
v = pan_verb.verbalize("ee46a040-d70f-11ed-8cc6-76349838619d", to_verbalize="documentID")
for n in v["verbalized"]:
    print(n)


This document uses more POS Unigram: CCONJ than the average
This document uses more POS Unigram: NUM than the average
This document uses more POS Bigram: ('AUX', 'VERB') than the average
This document uses more POS Bigram: ('NOUN', 'CCONJ') than the average
This document uses more Function word: i than the average
This document uses more Function word: its than the average
This document uses more Function word: were than the average
This document uses more Function word: being than the average
This document uses more Function word: an than the average
This document uses more Function word: but than the average
This document uses more Function word: by than the average
This document uses more Function word: over than the average
This document uses more Function word: some than the average
This document uses more Letter: o than the average
This document uses more Letter: x than the average
This document uses more Letter: N than the average
This document uses more Dependency label: auxpas

In [4]:
hrs_ids = {
    "authors":[
        "6e34df08-ed37-5941-85a3-d590511781f9",
        "2de9ffc8-deb7-53fb-b4db-a6ab9af0bbf1",
        "dd47dc76-ad48-5d40-b1c8-71191522f15d"
        ],
    "docs":[
        "322d6728-39de-54f1-9568-da4352aca6f7",
        "a4651175-6e96-5075-832f-fd818873f669",
        "574f9067-0f11-5753-8036-9d4112106301"
    ]
}

#hrs_verb = Verbalizer(datapaths["hrs"])

ValueError: ('Lengths must match to compare', (3138,), (1,))

['fff']